In [1]:
import pandas as pd
pd.DataFrame([1,2,3])

,0
0,1
1,2
2,3


In [7]:
import datetime
timePoint = datetime.datetime.today() - datetime.timedelta(days=1)
timePoint = timePoint.replace(hour=21, minute=00, second=00, microsecond=0)
timePoint

datetime.datetime(2018, 9, 24, 21, 0)

In [14]:
from pymongo import MongoClient
client = MongoClient("localhost",27017)
db = client["VnTrader_Tick_Db"]
items = db["rb1901"].find().limit(5)
for i in items:
    print i

{u'bidVolume5': 0, u'bidVolume4': 0, u'bidVolume3': 0, u'bidVolume2': 0, u'openPrice': 4158.0, u'datetime': datetime.datetime(2018, 9, 6, 21, 15, 44), u'askVolume1': 1543, u'askVolume3': 0, u'askVolume2': 0, u'askVolume5': 0, u'rawData': None, u'lastPrice': 4184.0, u'highPrice': 4190.0, u'openInterest': 2782988.0, u'preClosePrice': 4148.0, u'lowPrice': 4158.0, u'upperLimit': 4411.0, u'gatewayName': u'CTP', u'lowerLimit': 3834.0, u'exchange': u'SHFE', u'symbol': u'rb1901', u'volume': 676438, u'bidVolume1': 192, u'date': u'20180906', u'lastVolume': 0, u'askPrice4': 0.0, u'bidPrice5': 0.0, u'bidPrice4': 0.0, u'bidPrice1': 4184.0, u'bidPrice3': 0.0, u'bidPrice2': 0.0, u'vtSymbol': u'rb1901', u'time': u'21:15:44.0', u'askPrice5': 0.0, u'_id': ObjectId('5b912880659aa055848c5374'), u'askPrice3': 0.0, u'askPrice2': 0.0, u'askPrice1': 4185.0, u'askVolume4': 0}
{u'bidVolume5': 0, u'bidVolume4': 0, u'bidVolume3': 0, u'bidVolume2': 0, u'openPrice': 4158.0, u'datetime': datetime.datetime(2018, 9, 6

In [30]:
import json
def get_db(host,port,dbName):
    client = MongoClient(host,port)
    db = client[dbName]
    return db

def get_specificItems(db, coll_name):
    Items = db[coll_name].find().limit(5) 
    return Items

db = get_db("localhost", 27017, 'VnTrader_Tick_Db')
df = pd.DataFrame(list(get_specificItems(db, 'rb1901')))
del df["_id"]
df
df.dropna(axis=0, how='all')
data = json.loads(df.T.to_json(date_format = 'iso')).values()
data

[{u'askPrice1': 4185.0,
  u'askPrice2': 0.0,
  u'askPrice3': 0.0,
  u'askPrice4': 0.0,
  u'askPrice5': 0.0,
  u'askVolume1': 1558,
  u'askVolume2': 0,
  u'askVolume3': 0,
  u'askVolume4': 0,
  u'askVolume5': 0,
  u'bidPrice1': 4184.0,
  u'bidPrice2': 0.0,
  u'bidPrice3': 0.0,
  u'bidPrice4': 0.0,
  u'bidPrice5': 0.0,
  u'bidVolume1': 160,
  u'bidVolume2': 0,
  u'bidVolume3': 0,
  u'bidVolume4': 0,
  u'bidVolume5': 0,
  u'date': u'20180906',
  u'datetime': u'2018-09-06T21:15:45.000Z',
  u'exchange': u'SHFE',
  u'gatewayName': u'CTP',
  u'highPrice': 4190.0,
  u'lastPrice': 4184.0,
  u'lastVolume': 0,
  u'lowPrice': 4158.0,
  u'lowerLimit': 3834.0,
  u'openInterest': 2783026.0,
  u'openPrice': 4158.0,
  u'preClosePrice': 4148.0,
  u'rawData': None,
  u'symbol': u'rb1901',
  u'time': u'21:15:45.0',
  u'upperLimit': 4411.0,
  u'volume': 676518,
  u'vtSymbol': u'rb1901'},
 {u'askPrice1': 4185.0,
  u'askPrice2': 0.0,
  u'askPrice3': 0.0,
  u'askPrice4': 0.0,
  u'askPrice5': 0.0,
  u'askVolum

In [7]:
rb_period = "2010-01-01||8:59 - 10:15, 10:30 - 11:30, 13:30 - 15:00%2014-12-26||9:00 - 10:15, 10:30 - 11:30, 13:30 - 15:00, 20:59 - 00:00, 00:00 - 01:00"
phase = [i for i in rb_period.split('%')]  
phase.sort(reverse=True)
for i in phase:
    startDate = datetime.datetime.strptime(i.split('||')[0], "%Y-%m-%d")
    if startDate <= timePoint:
        return i.split('||')[1]
    else:
        continue
        

SyntaxError: 'return' outside function (<ipython-input-7-55db32129cea>, line 7)

In [95]:
from pymongo import MongoClient
import datetime
import pandas as pd

def get_db(host,port,dbName):
    # 建立连接，似乎同一个主机上port只能唯一
    client = MongoClient(host,port)
    db = client[dbName]
    return db

def get_all_colls(db):
    return [i for i in db.collection_names()]

def get_specificItems(db, coll_name, time):
    Items = db[coll_name].find({"datetime": {'$gte': time}},allow_partial_results=False) # 注：原始db的datetime为ISO格式需要转换才能比较
    return Items

timePoint = datetime.datetime.today() - datetime.timedelta(days=46)
timePoint = timePoint.replace(hour=21, minute=00, second=00, microsecond=0)

db = get_db("localhost", 27017, 'VnTrader_Tick_Db')
names = get_all_colls(db)
print names

def get_allItems(db, coll_name):
    Items = db[coll_name].find({}) # 注：原始db的datetime为ISO格式需要转换才能比较
    return Items

def get_oneItem(db, coll_name):
    Items = db[coll_name].find({}).limit(1) # 注：原始db的datetime为ISO格式需要转换才能比较
    return Items

# one = get_oneItem(db, 'rb1901')
cur = get_allItems(db, 'rb1901')
# col = pd.DataFrame(list(one)).columns
# print col                   
# df = pd.DataFrame(columns = col)  
# for i in cur:
#     pd_data=pd.DataFrame.from_dict(i,orient='index').T   
#     #插入df,忽略索引
#     df=df.append(pd_data, ignore_index=True)  
# print df.tail(3)
# print get_specificItems(db, names[1], timePoint).count()
# df = pd.DataFrame(list(get_specificItems(db, names[1], timePoint)))
# print df.head(3)
print cur.count()
print "cur type is %s"  % str(type(cur))
print "cur[0] type is %s"  % str(type(cur[0]))
print "cur[0:3] type is %s"  % str(type(cur[0:3]))
print list(cur)
# for i in cur[0:5]:
#     print i
ls = []
# def yd():
#     for i in cur:
#         yield i
        
# for i in yd():
#     ls.append(i)
# print ls[2]
# for i in xrange(8):
#     ls.append(i)
#     i+=1
# print ls[-1]

def read_file(fpath): 
    BLOCK_SIZE = 1024 
    with open(fpath, 'rb') as f: 
        while True: 
            block = f.read(BLOCK_SIZE) 
            if block: 
                yield block 
            else: 
                return

def load_mongodb(items):
    BLOCK_SIZE = 1024
    with items as f:
        while True:
            block = list(f)

[u'm1901', u'rb1901', u'SR.HOT', u'p1901', u'SR901', u'm.HOT', u'rb.HOT', u'l1901', u'l.HOT', u'p.HOT']
903536
cur type is <class 'pymongo.cursor.Cursor'>
cur[0] type is <type 'dict'>
cur[0:3] type is <class 'pymongo.cursor.Cursor'>


TypeError: 'list' object is not callable

In [76]:
import pandas as pd
dfInfo = pd.read_csv('E:\\Github\\dbProcess\\BasicInformation.csv')
dfInfo.index = dfInfo['Symbol'].tolist()
del dfInfo['Symbol']
dfInfo.loc[['rb'],['TradingPeriod']]
#phase = [i for i in dfInfo["TradingPeriod"].tolist().split('%')]
#dfInfo["TradingPeriod"].str.split('%')
temp = dfInfo.iloc[23,1].split('%')
temp1 = [i.split("||") for i in temp]
temp2 = [float(i[0].replace("-","")) for i in temp1]
tar = 20120101
dfInfo['dd']=999
# temp1
# format = lambda x: x > 20090101  
# filter(format,temp2)
# for i in dfInfo['TradingUnits']:
#     print i

In [20]:
import pandas as pd
import datetime as dt
rb = pd.read_csv('E:\\Github\\dbProcess\\rbdfdate.csv')
str = rb['currPeriod'][0]
str = "".join(str.split())
ls1 = str.split(",")
ls2 = [i.split("-") for i in ls1]
str2dt = lambda x: dt.datetime.strptime(x,"%H:%M")
ls3 = [map(str2dt,i) for i in ls2]
ls4 = [(i[1]-i[0]).total_seconds() for i in ls3]
ls5 = [i+86340 if i<0 else i for i in ls4]
sum(ls5)*2
ls5
# now = dt.datetime.now()
# hourago = now - datetime.timedelta(hours=0.85)
# yesterday = now - datetime.timedelta(days=1)
# timespan1 = now - hourago
# timespan2 = now - yesterday
# timespan2.total_seconds()

[4500.0, 3600.0, 5400.0, 10800.0, 3600.0]